In [5]:
import os

In [6]:
%pwd

#It gives the output as the pwd - present working directory:

'c:\\Users\\shiva\\OneDrive\\Desktop\\Work\\NLP\\Projects\\Text_Summarizer_Project\\research'

In [7]:
#Going back from current research folder to root folder where everything is present:
os.chdir("../")

In [8]:
%pwd

#Now we are inside our projects-root folder

'c:\\Users\\shiva\\OneDrive\\Desktop\\Work\\NLP\\Projects\\Text_Summarizer_Project'

Updating the Entity:

In [ ]:
#Creating Entity:  Defining the return type of the function:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)

#It is a data class: It defines a custom return type for any function:
class DataIngestionConfig:

    #Defining variables:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

Updating the configuration Manager:

In [ ]:
#Updating the configuration manager: 

# Import all the constants:
from src.textSummarizer.constants import *

# Import necessary file from commonly used functions:
from src.textSummarizer.utils.common import read_yaml, create_directories

In [11]:
#Creating a class of configuration Manager:
class ConfigurationManager:

    #Defining a constructor with following paramteres:
    def __init__(self,config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):

        #Obtaining the config from the file_path:
        self.config = read_yaml(config_filepath)
        
        #Obtaining the paramters from the file_path:
        self.params = read_yaml(params_filepath)

        #Creating a new artifact: root folder:
        create_directories([self.config.artifacts_root])

    #Defining a function to get the data ingestion config:
    def get_data_ingestion_config(self) -> DataIngestionConfig:

        #Obtaining all the values of the current object:
        config = self.config.data_ingestion

        #Creating a directory for the root directory of the current config:
        create_directories([config.root_dir])

        #Setting up all the values of class:
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )


        return data_ingestion_config

5. Updating the components:


In [ ]:
import os

#Package used to download the data:
import urllib.request as request

#Package used to unzip the data after downloading:
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [ ]:
#Defining the components:
class DataIngestion:

    #constructor:
    def __init__(self,config: DataIngestionConfig):

        #Initialising the config:
        self.config = config

    #A function to download the file:
    def download_file(self):

        #If the file does not already exist in the file location:
        if not os.path.exists(self.config.local_data_file):

            #Obraining the filename and headers:
            filename, headers = request.urlretrieve(

                #Setting up the url:
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )

            #Logging that the file has be downloaded:
            logger.info(f"{filename} download! with following info: \n{headers}")

        #If the file already exist in the file location:
        else:

            #Logging that the file already exist with its size:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    #A function to Extract the zip file:
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data dictionary
        Function returns None
        """

        #Obtaining the unzip path:
        unzip_path = self.config.unzip_dir

        #Creating the unzip directory:
        os.makedirs(unzip_path,exist_ok=True)

        #Extracting zip file and uploading inside my unzip folder:
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)

6. Updating the Pipeline:

In [14]:
try:
    #Creating a configuration manager object:
    config = ConfigurationManager()

    #Obtaining the data ingestion components:
    data_ingestion_config = config.get_data_ingestion_config()

    #Calling the data ingestion components: Intialising the components of all the data ingestion by initialising an object:
    data_ingestion = DataIngestion(config=data_ingestion_config)

    #Downloading the file:
    data_ingestion.download_file()

    #Extracting the zip_file:
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e


[2025-03-27 01:47:08,009: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-27 01:47:08,033: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-27 01:47:08,038: INFO: common: created directory at: artifacts]
[2025-03-27 01:47:08,040: INFO: common: created directory at: artifacts/data_ingestion]
[2025-03-27 01:47:13,548: INFO: 2590737820: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 3C4C:2A8E62:935D6:14E32E:67E460C5
Accept-Ranges: bytes
Date: Wed, 26 Mar 2025 20:17:10 GMT
Via: 1.1 varnish
X-Served-By: cache-del21721-DEL
X-Cache: MI